In [1]:
import os, json
from dotenv import load_dotenv
from pymongo import MongoClient

# --- Connect ---
load_dotenv()
client = MongoClient(os.getenv("MONGODB_URI"))
coll = client[os.getenv("MONGODB_DATABASE")]["inventories"]

# --- PARAMETERS ---
ITEM_BARCODE = "4809992000101"
ADDSTOCK =12
SOLD = 34

def _num(n):
    if isinstance(n, (int, float)): return float(n)
    try: return float(str(n).replace(",", ""))
    except Exception: raise ValueError("ADDSTOCK/SOLD must be numbers")

ADDSTOCK = _num(ADDSTOCK); SOLD = _num(SOLD)
if ADDSTOCK < 0 or SOLD < 0: raise ValueError("ADDSTOCK and SOLD must be non-negative")

# --- Identify the ONLY open form (is_empty=True) ---
open_forms = list(coll.find({"is_empty": True}, {"_id": 1, "id": 1, "current_form_qr_code": 1}).limit(2))
if len(open_forms) == 0:
    raise RuntimeError("No inventory document has is_empty=True.")
if len(open_forms) > 1:
    raise RuntimeError("More than one inventory document has is_empty=True; cannot determine target uniquely.")

target_oid = open_forms[0]["_id"]
target_id = open_forms[0].get("id")
target_qr = open_forms[0].get("current_form_qr_code")

# --- Update ONLY the matching item fields; DO NOT touch is_empty ---
pipeline_update = [
    {
        "$set": {
            "items": {
                "$map": {
                    "input": "$items",
                    "as": "it",
                    "in": {
                        "$cond": [
                            {"$eq": ["$$it.barcode", ITEM_BARCODE]},
                            {
                                "$mergeObjects": [
                                    "$$it",
                                    {
                                        "addstock": ADDSTOCK,
                                        "sold": SOLD,
                                        "updated": "$$NOW",
                                        "current_quantity": {
                                            "$subtract": [
                                                {"$add": ["$$it.previous_quantity", ADDSTOCK]},
                                                SOLD
                                            ]
                                        }
                                    }
                                ]
                            },
                            "$$it"
                        ]
                    }
                }
            }
        }
    }
]

res = coll.update_one(
    {"_id": target_oid, "is_empty": True, "items.barcode": ITEM_BARCODE},
    pipeline_update
)
if res.matched_count == 0:
    raise RuntimeError("The open form has no item with the given barcode (or it is no longer is_empty=True).")

# --- Read back the updated item ---
doc = coll.find_one({"_id": target_oid}, {"_id": 0})
item = next((it for it in doc.get("items", []) if it.get("barcode") == ITEM_BARCODE), None)
if not item:
    raise RuntimeError("Updated item not found after write (unexpected).")

balanced = (item["previous_quantity"] + item["addstock"] - item["sold"]) == item["current_quantity"]

print(json.dumps({
    "ok": True,
    "inventory_id": doc.get("id", target_id),
    "current_form_qr_code": doc.get("current_form_qr_code", target_qr),
    "is_empty": doc.get("is_empty"),  # unchanged
    "item": {
   
        "barcode": item["barcode"],
        "previous_quantity": item["previous_quantity"],
        "addstock": item["addstock"],
        "sold": item["sold"],
        "current_quantity": item["current_quantity"],
        "updated": item.get("updated"),
    },
    "balance_ok": balanced
}, ensure_ascii=False, indent=2, default=str))


{
  "ok": true,
  "inventory_id": "edfa2b5e-657e-4616-990b-1b69c983ee86",
  "current_form_qr_code": "87654321",
  "is_empty": true,
  "item": {
    "barcode": "4809992000101",
    "previous_quantity": 570,
    "addstock": 12.0,
    "sold": 34.0,
    "current_quantity": 548.0,
    "updated": "2025-10-14 16:29:07.074000"
  },
  "balance_ok": true
}
